In [1]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
import pytz

In [2]:
import os
import numpy as np
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/master project/data/MSFT' #เข้า drive

Mounted at /content/drive


In [3]:
api_key = 'J4AM5YKF88UA41WO'

In [4]:
def _get_data(symbols, time_from, time_to, api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

In [5]:
def get_dataset(time_from="20190101T0000",
                time_to="20250301T0000",
                MAX_API_CALLS_PER_DAY = 99999, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 300 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%600==0:
            time.sleep(65)

        data=_get_data('MSFT',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract MSFT specific relevance (we didn't use it in video)

    # Only take tickers with MSFT in headline
    df=df[df.title.str.contains('MSFT|Microsoft',case=False)]
    # Only take when # of tickers = 1
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    #df.set_index('time_published',inplace=True)
    # Sort by time published
    #df.sort_index(inplace=True)
    return df


In [6]:
df = get_dataset()

In [7]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.tz_localize('UTC')

In [8]:
df['time_published']=df['time_published'].dt.tz_convert('US/Eastern')

In [9]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.strftime('%Y-%m-%d')

In [10]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
4,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,https://www.benzinga.com/25/02/44053318/nvidia...,2025-02-28,[Surbhi Jain],"Nvidia-backed CoreWeave targets $4B IPO, eyein...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'IPO', 'relevance_score': '0.576289...",0.102035,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.2121..."
9,Why Is Microsoft ( MSFT ) Down 5.4% Since La...,https://www.zacks.com/stock/news/2423780/why-i...,2025-02-28,[Zacks Equity Research],Microsoft (MSFT) reported earnings 30 days ago...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.178840,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.4421..."
11,Performance Comparison: Microsoft And Competit...,https://www.benzinga.com/insights/news/25/02/4...,2025-02-28,[Benzinga Insights],In today's rapidly changing and fiercely compe...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.250313,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.4834..."
21,A Once-in-a-Lifetime Market Opportunity: Is Al...,https://www.fool.com/investing/2025/02/28/a-on...,2025-02-28,[Keithen Drury],Similar to how artificial intelligence ( AI ) ...,https://media.ycharts.com/charts/1da9237c7a2e2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.230358,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.2727..."
25,Microsoft Urges Trump To Avoid This Critical M...,https://www.benzinga.com/25/02/44035813/micros...,2025-02-28,[Namrata Sen],Microsoft Corporation MSFT has cautioned the T...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.108406,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.3317..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,"Why Amazon, Meta Platforms And Microsoft Share...",https://www.benzinga.com/markets/cryptocurrenc...,2022-03-07,[Henry Khederian],Shares of several companies in the broader tec...,https://cdn2.benzinga.com/files/imagecache/og_...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",-0.116559,Neutral,"[{'ticker': 'FB', 'relevance_score': '0.550151..."
327,"Apple, Google, Microsoft, and other tech giant...",https://www.fastcompany.com/90728462/apple-goo...,2022-03-07,[],Return-to-office update: list of tech companie...,https://images.fastcompany.net/image/upload/w_...,Fast Company,n/a,www.fastcompany.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.245551,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.159463..."
333,Microsoft announces setting up of its fourth d...,https://www.business-standard.com/article/comp...,2022-03-07,[Shivani Shinde],"Situated in Hyderabad, this will be the Micros...",https://bsmedia.business-standard.com/_media/b...,Business Standard,Companies,www.business-standard.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.085910,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.6560..."
335,Microsoft unveils fourth data center in India,https://www.moneycontrol.com/news/business/mic...,2022-03-07,[],Microsoft India head Anant Maheshwari said the...,https://images.moneycontrol.com/static-mcnews/...,Money Control,Business,www.moneycontrol.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.070558,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.4200..."


In [11]:
News_MSFT = df[["time_published", "title", "summary","overall_sentiment_score"]]

In [12]:
News_MSFT

,time_published,title,summary,overall_sentiment_score
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035
9,2025-02-28,Why Is Microsoft ( MSFT ) Down 5.4% Since La...,Microsoft (MSFT) reported earnings 30 days ago...,0.178840
11,2025-02-28,Performance Comparison: Microsoft And Competit...,In today's rapidly changing and fiercely compe...,0.250313
21,2025-02-28,A Once-in-a-Lifetime Market Opportunity: Is Al...,Similar to how artificial intelligence ( AI ) ...,0.230358
25,2025-02-28,Microsoft Urges Trump To Avoid This Critical M...,Microsoft Corporation MSFT has cautioned the T...,0.108406
...,...,...,...,...
313,2022-03-07,"Why Amazon, Meta Platforms And Microsoft Share...",Shares of several companies in the broader tec...,-0.116559
327,2022-03-07,"Apple, Google, Microsoft, and other tech giant...",Return-to-office update: list of tech companie...,-0.245551
333,2022-03-07,Microsoft announces setting up of its fourth d...,"Situated in Hyderabad, this will be the Micros...",0.085910
335,2022-03-07,Microsoft unveils fourth data center in India,Microsoft India head Anant Maheshwari said the...,0.070558


In [13]:
!pip install nltk

In [14]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [15]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
sia = SentimentIntensityAnalyzer()
# Analyze sentiment for each summary

x = str(News_MSFT['summary']).encode("utf-8")
News_MSFT['sentiment_score'] = News_MSFT['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])

<ipython-input-16-9cb2628f53e4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  News_MSFT['sentiment_score'] = News_MSFT['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [17]:
News_MSFT

,time_published,title,summary,overall_sentiment_score,sentiment_score
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035,0.5106
9,2025-02-28,Why Is Microsoft ( MSFT ) Down 5.4% Since La...,Microsoft (MSFT) reported earnings 30 days ago...,0.178840,0.0000
11,2025-02-28,Performance Comparison: Microsoft And Competit...,In today's rapidly changing and fiercely compe...,0.250313,0.7783
21,2025-02-28,A Once-in-a-Lifetime Market Opportunity: Is Al...,Similar to how artificial intelligence ( AI ) ...,0.230358,0.4767
25,2025-02-28,Microsoft Urges Trump To Avoid This Critical M...,Microsoft Corporation MSFT has cautioned the T...,0.108406,0.3182
...,...,...,...,...,...
313,2022-03-07,"Why Amazon, Meta Platforms And Microsoft Share...",Shares of several companies in the broader tec...,-0.116559,-0.3182
327,2022-03-07,"Apple, Google, Microsoft, and other tech giant...",Return-to-office update: list of tech companie...,-0.245551,0.0000
333,2022-03-07,Microsoft announces setting up of its fourth d...,"Situated in Hyderabad, this will be the Micros...",0.085910,0.0000
335,2022-03-07,Microsoft unveils fourth data center in India,Microsoft India head Anant Maheshwari said the...,0.070558,0.0000


In [18]:
News_MSFT_Nozero=News_MSFT.loc[News_MSFT['sentiment_score'] != 0]
News_MSFT_Nozero

,time_published,title,summary,overall_sentiment_score,sentiment_score
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035,0.5106
11,2025-02-28,Performance Comparison: Microsoft And Competit...,In today's rapidly changing and fiercely compe...,0.250313,0.7783
21,2025-02-28,A Once-in-a-Lifetime Market Opportunity: Is Al...,Similar to how artificial intelligence ( AI ) ...,0.230358,0.4767
25,2025-02-28,Microsoft Urges Trump To Avoid This Critical M...,Microsoft Corporation MSFT has cautioned the T...,0.108406,0.3182
26,2025-02-28,"Activist Investor ValueAct, Once Linked To Ste...","Activist investor ValueAct Holdings, L.P., kno...",0.288768,0.4588
...,...,...,...,...,...
244,2022-03-08,This Steel Company Has A Better 1-Year Return ...,United States Steel Corporation ( NYSE: X ) ...,0.054233,0.8268
254,2022-03-08,"Betting on Indian digital market, Microsoft un...",The overall Indian public cloud services marke...,0.075132,0.0258
309,2022-03-07,MSFT: Scoop Up These 3 Dow Jones Stocks Down M...,The stock market has been rattled since the be...,-0.014077,-0.8176
313,2022-03-07,"Why Amazon, Meta Platforms And Microsoft Share...",Shares of several companies in the broader tec...,-0.116559,-0.3182


In [19]:
daily_avg_sentiment_MSFT = News_MSFT.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_MSFT

,time_published,sentiment_score
0,2022-03-01,0.926000
1,2022-03-07,-0.189300
2,2022-03-08,0.180000
3,2022-03-09,0.403750
4,2022-03-10,-0.180283
...,...,...
1038,2025-02-24,0.290438
1039,2025-02-25,0.135475
1040,2025-02-26,0.415457
1041,2025-02-27,0.451543


In [20]:
daily_avg_sentiment_Nozero_MSFT = News_MSFT_Nozero.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_Nozero_MSFT

,time_published,sentiment_score
0,2022-03-01,0.926000
1,2022-03-07,-0.567900
2,2022-03-08,0.330000
3,2022-03-09,0.538333
4,2022-03-10,-0.270425
...,...,...
1009,2025-02-24,0.387250
1010,2025-02-25,0.154829
1011,2025-02-26,0.581640
1012,2025-02-27,0.451543


In [21]:
daily_avg_sentiment_MSFT.to_csv(os.path.join(path,'daily_avg_sentiment_MSFT.csv'))
News_MSFT.to_csv(os.path.join(path,'News_MSFT.csv'))

In [22]:
daily_avg_sentiment_Nozero_MSFT.to_csv(os.path.join(path,'daily_avg_sentiment_Nozero_MSFT.csv'))
News_MSFT_Nozero.to_csv(os.path.join(path,'News_MSFT_Nozero.csv'))